# Import thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import traceback

In [2]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument("--start-maximized")
s = Service('C:\chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(0)

In [3]:
# List tất cả url movie
list_all_url_movie = pd.read_csv('E:/Crawl_Web/Rotten Tomatoes/Crawl URL/URL_movie.csv').squeeze('columns').tolist()

In [4]:
df_movie = []

for url_movie in list_all_url_movie[-10000:]:
    # Mở url phim
    driver.get(url_movie)
    
    try:
        # Đợi cho đến khi url load xong
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
        soup = BeautifulSoup(html_of_interest, 'lxml')

        ## Tạo Dict chứa thông tin về phim
        dict_info_movie = {}

        # Tiêu đề phim
        dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text

        # Điểm đánh giá của nhà phê bình
        dict_info_movie['Tomatometer score'] = soup.select('score-board')[0].attrs['tomatometerscore']

        # Điểm đánh giá của khán giả
        dict_info_movie['Audience score'] = soup.select('score-board')[0].attrs['audiencescore']

        # Số lượt đánh giá của nhà phê bình
        dict_info_movie['Tomatometer count'] = soup.select('a[slot="critics-count"]')[0].text.strip()

        # Số lượt đánh giá của khán giả
        dict_info_movie['Audience count'] = soup.select('a[slot="audience-count"]')[0].text.strip()

        # Trạng thái đánh giá phim của nhà phê bình
        dict_info_movie['Tomatometer state'] = soup.select('score-board')[0].attrs['tomatometerstate']

        # Trạng thái đánh giá phim của khán giả
        dict_info_movie['Audience state'] = soup.select('score-board')[0].attrs['audiencestate']

        # Thông tin phim
        list_name_info = soup.select('div[class="meta-label subtle"]')
        list_info = soup.select('div[data-qa="movie-info-item-value"]')

        for num_info in range(len(list_name_info)):
            name_info = list_name_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            info = list_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            dict_info_movie[name_info] = info

        # Tóm tắt nội dụng phim
        dict_info_movie['Synopsis'] = soup.select('div[id="movieSynopsis"]')[0].text.strip()

        # Diễn viên
        list_raw_cast = soup.select('span[class="characters subtle smaller"]')
        list_cast = []

        for c in list_raw_cast:
            list_cast.append(c.attrs['title'])

        dict_info_movie['Cast'] = ', '.join(list_cast)

        # Reviews của nhà phê bình
        driver.get(url_movie + '/reviews')

        element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="layout reviews-page-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
        soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

        # Check có review của nhà phê bình không ?
        check_critic_review = soup_critic_review.select('div[class="review_table"]')[0].text.strip()

        if check_critic_review != '':
            list_critic_review = []
            list_top_critic_review = []
            
            # Số lượng review trong 1 trang
            num_rv = len(soup_critic_review.select('div[class="review-row"]'))

            for num in range(num_rv):
                rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text
                list_critic_review.append(rv)

                # Check top critic reviews
                check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                if check_top_critic != []:
                    list_top_critic_review.append(rv)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="review_table"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
                    soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

                    num_rv = len(soup_critic_review.select('div[class="review-row"]'))

                    for num in range(num_rv):
                        rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text.strip()
                        list_critic_review.append(rv)

                        check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                        if check_top_critic != []:
                            list_top_critic_review.append(rv)
                    
                except:
                    break
            dict_info_movie['Critic review'] = '<>'.join(list_critic_review)
            if list_top_critic_review != []:
                dict_info_movie['Top critic review'] = '<>'.join(list_top_critic_review)
            else:
                dict_info_movie['Top critic review'] = 'No top critic reviews'
        elif check_critic_review == '':
            dict_info_movie['Critic review'] = 'No critic reviews'
            dict_info_movie['Top critic review'] = 'No top critic reviews'

        # Reviews của khán giả
        driver.get(url_movie + '/reviews?type=user')

        element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-qa="reviews-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
        soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

        ## Check có review của khán giả không ?
        check_audience_review = soup_audience_review.select('div[id="movieUserReviewsContent"]')

        if len(check_audience_review) != 0:
            list_audience_review = []
            list_verified_audience_review = []
            
            # Số lượng review trong 1 trang
            num_audience_rv = len(soup_audience_review.select('div[class="audience-review-row"]'))
            
            for num in range(num_audience_rv):
                rv = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('p', {'data-qa': 'review-text'})[0].text
                list_audience_review.append(rv)

                # Check verified audience review
                check_verified_audience = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('span', {'class': 'audience-reviews__verified'})

                if check_verified_audience != []:
                    list_verified_audience_review.append(rv)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="movieUserReviewsContent"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
                    soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

                    # Số lượng review trong 1 trang
                    num_audience_rv = len(soup_audience_review.select('div[class="audience-review-row"]'))
                    
                    for num in range(num_audience_rv):
                        rv = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('p', {'data-qa': 'review-text'})[0].text
                        list_audience_review.append(rv)

                        # Check verified audience review
                        check_verified_audience = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('span', {'class': 'audience-reviews__verified'})

                        if check_verified_audience != []:
                            list_verified_audience_review.append(rv)
                    
                except:
                    break
            dict_info_movie['Audience review'] = '<>'.join(list_audience_review)
            if list_verified_audience_review != []:
                dict_info_movie['Verified audience review'] = '<>'.join(list_verified_audience_review)
            else:
                dict_info_movie['Verified audience review'] = 'No verified audience reviews'

        else:
            dict_info_movie['Audience review'] = 'No audience reviews'
            dict_info_movie['Verified audience review'] = 'No verified audience reviews'

        df_movie.append(dict_info_movie)

    except Exception:
        # traceback.print_exc()
        print(url_movie)
        continue
    
df_movie = pd.DataFrame(df_movie)

https://www.rottentomatoes.com/m/a-christmas-reunion
https://www.rottentomatoes.com/m/vis_a_vis_2014
https://www.rottentomatoes.com/m/heathens_and_thieves_2012
https://www.rottentomatoes.com/m/annual_ann_arbor_film_festival
https://www.rottentomatoes.com/m/the_nada_gang
https://www.rottentomatoes.com/m/sin_muertos_no_hay_carnaval
https://www.rottentomatoes.com/m/the_laplaces_demon
https://www.rottentomatoes.com/m/the_royal_ballet_woolf_works
https://www.rottentomatoes.com/m/i-due-foscari
https://www.rottentomatoes.com/m/gaming_the_real_world
https://www.rottentomatoes.com/m/vasha
https://www.rottentomatoes.com/m/sundance_film_festival_shorts
https://www.rottentomatoes.com/m/english_national_opera_la_traviata
https://www.rottentomatoes.com/m/treintona_soltera_y_fantastica
https://www.rottentomatoes.com/m/rifftrax_live_christmas_shorts_stravaganza
https://www.rottentomatoes.com/m/den_brother
https://www.rottentomatoes.com/m/boys_of_summerville
https://www.rottentomatoes.com/m/imitation_g

In [5]:
df_movie

,Title,Tomatometer score,Audience score,Tomatometer count,Audience count,Tomatometer state,Audience state,Genre,Original Language,Director,...,Synopsis,Cast,Rating,Box Office (Gross USA),Release Date (Streaming),Release Date (Theaters),Distributor,Sound Mix,Aspect Ratio,View the collection
0,The Wishful Thinkers,,,0 Reviews,Fewer than 50 Ratings,,,Drama,Spanish,Jonás Trueba,...,Filmmakers waste time when they are not making...,"Aura Garrido, Isabelle Stoffel, Francesco Carr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Loyalties,,,0 Reviews,0 Ratings,,,Drama,English,Anne Wheeler,...,The upper-class wife (Susan Wooldridge) of a p...,"Susan Wooldridge, Kenneth Welsh, Tantoo Cardin...",R,$1.3K,NaN,NaN,NaN,NaN,NaN,NaN
2,Dirty War,,,0 Reviews,0 Ratings,,,Action,Spanish,Juan Piquer Simon,...,"Disgusted by his employers' brutal tactics, a ...","Frank Braña, Mariano Vidal Molina, Pierre Oudr...",R,NaN,"Sep 1, 2016",NaN,NaN,NaN,NaN,NaN
3,Twilight of the Dark Master,,49,0 Reviews,"1,000+ Ratings",,spilled,"Fantasy, Anime",Japanese,Akiyuki Shinbo,...,"In the near future, demons prepare for a new a...","Andrew Philpot, Toshihiko Seki, Masuo Amada, S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lemony Snicket's A Series of Unfortunate Events,72,63,161 Reviews,"250,000+ Ratings",fresh,upright,"Kids & family, Comedy, Fantasy, Adventure",English,,...,After the three young Baudelaire siblings are ...,"Jim Carrey, Liam Aiken, Emily Browning, Kara H...",PG (Scary Situations|Brief Language|Thematic E...,$118.6M,"Apr 26, 2005","Dec 17, 2004 wide",DreamWorks SKG,"Surround, Dolby SRD, DTS, SDDS",Flat (1.371),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,Northeast,20,17,5 Reviews,50+ Ratings,rotten,spilled,Drama,English,Gregory Kohn,...,"Will, an aimless playboy living in Brooklyn, h...","David Call, Veronika Dominczyk, Tate Ellington...",NaN,NaN,"Dec 10, 2012",NaN,NaN,NaN,NaN,NaN
9964,Penny Paradise,,,0 Reviews,Fewer than 50 Ratings,,,Comedy,English,Carol Reed,...,The skipper of a tugboat comes to regret jumpi...,"Edmund Gwenn, Betty Driver, Jimmy O'Dea, Ethel...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9965,Freak City,,74,0 Reviews,100+ Ratings,,upright,Drama,English,Lynne Littman,...,A woman (Samantha Mathis) with multiple sclero...,"Samantha Mathis, Marlee Matlin, Natalie Cole, ...",TV14 (S|L|D|V),NaN,"Jul 26, 2020",NaN,NaN,NaN,NaN,NaN
9966,Day of Atonement,,40,1 Reviews,Fewer than 50 Ratings,,spilled,Mystery & thriller,English,Alexandre Arcady,...,"An ex-crime boss (Christopher Walken), just re...","Christopher Walken, Jennifer Beals, Jill Clayb...",R,NaN,NaN,NaN,NaN,Surround,NaN,NaN


In [6]:
df_movie.to_csv('Crawl_10000_movie_no_reviews.csv', index=False)